# Подключение библиотек

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from datetime import date
from pyspark.sql import Window, Row
from user_agents import parse

# Формирование нового файла данных с информацией об устройстве и браузеру
# на основе анализа User_agent (https://pypi.org/project/user-agents/)

In [14]:
import csv
from csv import reader

new_file = open('access.log.m','w+')
csv_writer = csv.writer(new_file,delimiter=' ')
with open('access.log','r') as file:
    csv_reader = reader(file,delimiter=' ')
    for line in csv_reader:
        ua_string = line[9]
        user_agent = parse(ua_string)
        line.append(user_agent.browser.family)
        line.append(user_agent.device.brand)
        csv_writer.writerow(line)

# Создание SparkSession

In [20]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

# Создание дата фрейма лог файла

In [4]:
schema = T.StructType(fields=[
    T.StructField("IP", T.StringType(), True),
    T.StructField("sign_1", T.StringType(), True),
    T.StructField("sign_2", T.StringType(), True),
    T.StructField("Date_access", T.StringType(), True),
    T.StructField("Date_access_", T.StringType(), True),
    T.StructField("Action", T.StringType(), True),
    T.StructField("Status", T.IntegerType(), True),
    T.StructField("Size", T.IntegerType(), True),
    T.StructField("sign_3", T.StringType(), True),
    T.StructField("User_agent", T.StringType(), True),
    T.StructField("sign_4", T.StringType(), True),
    
    T.StructField("Browser", T.StringType(), True),
    T.StructField("Device", T.StringType(), True),
])

In [5]:
df = spark.read.csv("access.log.m", schema=schema, sep=" ")


In [19]:
df.show()

+-------------+------+------+--------------------+------------+--------------------+------+-----+--------------------+--------------------+------+-------------+------+
|           IP|sign_1|sign_2|         Date_access|Date_access_|              Action|Status| Size|              sign_3|          User_agent|sign_4|      Browser|Device|
+-------------+------+------+--------------------+------------+--------------------+------+-----+--------------------+--------------------+------+-------------+------+
| 54.36.149.41|     -|     -|[22/Jan/2019:03:5...|      +0330]|GET /filter/27|13...|   200|30577|                   -|Mozilla/5.0 (comp...|     -|    AhrefsBot|Spider|
|  31.56.96.51|     -|     -|[22/Jan/2019:03:5...|      +0330]|GET /image/60844/...|   200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|     -|Chrome Mobile|Huawei|
|  31.56.96.51|     -|     -|[22/Jan/2019:03:5...|      +0330]|GET /image/61474/...|   200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|     -|Chrome Mobile|

# Проверка содержимого столбцов

In [6]:
df.count()

10365152

In [9]:
df.groupby("IP")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+---------------+------+
|             IP| activ|
+---------------+------+
|  66.249.66.194|353483|
|   66.249.66.91|314522|
|151.239.241.163| 92475|
|   66.249.66.92| 88332|
|    91.99.30.32| 45979|
|  104.222.32.91| 42058|
|    91.99.72.15| 38694|
|    91.99.47.57| 38612|
|   5.78.190.233| 37204|
|195.181.168.181| 27979|
|   23.101.169.3| 27800|
|195.181.168.164| 24044|
|   66.249.66.93| 22146|
|   17.58.102.43| 21698|
|  104.222.32.94| 16706|
|   5.160.157.20| 13526|
|  5.117.116.238| 13427|
|   5.160.221.38| 12058|
|   172.20.2.174| 11303|
|  40.77.167.156| 10231|
+---------------+------+
only showing top 20 rows



In [8]:
 df.groupby("IP")\
    .agg(F.count("*").alias("activ"))\
    .orderBy("activ", ascending = False)\
    .agg(F.sum("activ"))\
    .show()

+----------+
|sum(activ)|
+----------+
|  10365152|
+----------+



In [10]:
df.groupby("sign_1")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+------+--------+
|sign_1|   activ|
+------+--------+
|     -|10365152|
+------+--------+



In [11]:
df.groupby("sign_2")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+------+--------+
|sign_2|   activ|
+------+--------+
|     -|10365150|
| admin|       2|
+------+--------+



In [12]:
df.groupby("sign_3")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+--------------------+-------+
|              sign_3|  activ|
+--------------------+-------+
|                   -|1646359|
|https://www.zanbi...| 494757|
|https://znbl.ir/s...| 387201|
|https://www.zanbi...| 219555|
|https://www.zanbi...| 128879|
|https://www.googl...|  98420|
|https://www.zanbi...|  87726|
|https://www.zanbi...|  79234|
|https://www.zanbi...|  74759|
|https://www.zanbi...|  59470|
|https://www.zanbi...|  58560|
|https://www.zanbi...|  47157|
|https://www.zanbi...|  47099|
|https://www.zanbi...|  44634|
|https://www.zanbi...|  43465|
|https://www.zanbi...|  38379|
|https://www.zanbi...|  37637|
|https://www.zanbi...|  31163|
|https://www.zanbi...|  30780|
|https://www.zanbi...|  30009|
+--------------------+-------+
only showing top 20 rows



In [13]:
df.groupby("sign_3")\
    .agg(F.count("*").alias("activ"))\
    .orderBy("activ", ascending = False)\
    .agg(F.sum("activ"))\
    .show()

+----------+
|sum(activ)|
+----------+
|  10365152|
+----------+



In [14]:
df.groupby("sign_4")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+--------------+--------+
|        sign_4|   activ|
+--------------+--------+
|             -|10341086|
|  5.127.61.214|     526|
|  51.38.89.157|     464|
|  5.112.171.19|     365|
|  5.218.20.237|     263|
|  46.51.51.189|     262|
|  5.134.139.78|     229|
|  83.121.197.9|     224|
|   5.218.73.35|     223|
| 185.246.4.210|     221|
|  5.120.175.61|     207|
| 89.198.135.67|     195|
| 46.209.15.186|     176|
|109.202.101.52|     165|
| 5.134.188.153|     163|
|   5.212.3.136|     162|
|  5.116.42.218|     160|
|    204.18.7.7|     158|
|  5.123.210.42|     145|
| 5.208.184.113|     136|
+--------------+--------+
only showing top 20 rows



In [15]:
df.groupby("sign_4")\
    .agg(F.count("*").alias("activ"))\
    .orderBy("activ", ascending = False)\
    .agg(F.sum("activ"))\
    .show()

+----------+
|sum(activ)|
+----------+
|  10365152|
+----------+



In [16]:
df.groupby("Date_access_")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+------------+--------+
|Date_access_|   activ|
+------------+--------+
|      +0330]|10365152|
+------------+--------+



# Убираем неинформативные столбцы. Получаем базовую таблицу

In [17]:
df_Base = df.drop("sign_1", "sign_2", "Date_access_", "sign_3", "sign_4")
df_Base.show()

+-------------+--------------------+--------------------+------+-----+--------------------+-------------+------+
|           IP|         Date_access|              Action|Status| Size|          User_agent|      Browser|Device|
+-------------+--------------------+--------------------+------+-----+--------------------+-------------+------+
| 54.36.149.41|[22/Jan/2019:03:5...|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|    AhrefsBot|Spider|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/60844/...|   200| 5667|Mozilla/5.0 (Linu...|Chrome Mobile|Huawei|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/61474/...|   200| 5379|Mozilla/5.0 (Linu...|Chrome Mobile|Huawei|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/14925/...|   200| 1696|Mozilla/5.0 (comp...|      bingbot|Spider|
|  91.99.72.15|[22/Jan/2019:03:5...|GET /product/3189...|   200|41483|Mozilla/5.0 (Wind...|      Firefox|  null|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/23488/...|   200| 2654|Mozilla/5.0 (comp...|     

# Заменяем нулевые значения дата-фрейма на "other"

In [18]:
df_Base = df_Base.na.fill("other")
df_Base.show()

+-------------+--------------------+--------------------+------+-----+--------------------+-------------+------+
|           IP|         Date_access|              Action|Status| Size|          User_agent|      Browser|Device|
+-------------+--------------------+--------------------+------+-----+--------------------+-------------+------+
| 54.36.149.41|[22/Jan/2019:03:5...|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|    AhrefsBot|Spider|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/60844/...|   200| 5667|Mozilla/5.0 (Linu...|Chrome Mobile|Huawei|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/61474/...|   200| 5379|Mozilla/5.0 (Linu...|Chrome Mobile|Huawei|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/14925/...|   200| 1696|Mozilla/5.0 (comp...|      bingbot|Spider|
|  91.99.72.15|[22/Jan/2019:03:5...|GET /product/3189...|   200|41483|Mozilla/5.0 (Wind...|      Firefox| other|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/23488/...|   200| 2654|Mozilla/5.0 (comp...|     

# Формирование таблицы 1 "Устройства по пользователям"

In [19]:
df_Devices = df_Base.groupby('Device')\
             .agg(F.count('IP').alias('Count_Users'))\
             .withColumn('Count_Users',F.col('Count_Users'))\
             .orderBy('Count_Users', ascending = False)
             
df_Devices.show()

+--------------------+-----------+
|              Device|Count_Users|
+--------------------+-----------+
|               other|    5204250|
|             Samsung|    1845565|
|              Spider|    1109418|
|               Apple|     742633|
|              Huawei|     555636|
|             Generic|     176737|
|                  LG|     173070|
|     Generic_Android|     140957|
|                Sony|      78339|
|                 HTC|      74043|
|              Lenovo|      59090|
|              XiaoMi|      54380|
|Generic_Android_T...|      47866|
|               Nokia|      38581|
|                Asus|      29729|
|        SonyEricsson|      14810|
|            Micromax|       5022|
|            Motorola|       3172|
|                 ZTE|       2346|
|               Tecno|       2262|
+--------------------+-----------+
only showing top 20 rows



In [22]:
df_Devices_Users = df_Devices.withColumn('All_Users', F.sum('Count_Users').over(Window.partitionBy()))\
            .withColumn('Ratio_Users', F.round(F.col('Count_Users') / F.col('All_Users'), 5))\
            .select('Device', 'Count_Users', 'Ratio_Users')
       
df_Devices_Users.show()


23/01/23 21:31:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-----------+-----------+
|              Device|Count_Users|Ratio_Users|
+--------------------+-----------+-----------+
|               other|    5204250|    0.50209|
|             Samsung|    1845565|    0.17805|
|               Apple|     742633|    0.07165|
|              Huawei|     555636|    0.05361|
|             Generic|     176737|    0.01705|
|                  LG|     173070|     0.0167|
|                Sony|      78339|    0.00756|
|                 HTC|      74043|    0.00714|
|              Lenovo|      59090|     0.0057|
|              XiaoMi|      54380|    0.00525|
|Generic_Android_T...|      47866|    0.00462|
|               Nokia|      38581|    0.00372|
|                Asus|      29729|    0.00287|
|        SonyEricsson|      14810|    0.00143|
|            Micromax|       5022|     4.8E-4|
|               Tecno|       2262|     2.2E-4|
|           Microsoft|       1302|     1.3E-4|
|             Alcatel|        975|     9.0E-5|
|           P

# Формирование таблицы 2 "Устройства по действиям"

In [23]:
df_Devices_ = df_Base.groupby('Device')\
             .agg(F.count('Action').alias('Count_Actions'))\
             .withColumn('Count_Actions',F.col('Count_Actions'))\
             .orderBy('Count_Actions', ascending = False)
             
df_Devices_.show()

+--------------------+-------------+
|              Device|Count_Actions|
+--------------------+-------------+
|               other|      5204250|
|             Samsung|      1845565|
|              Spider|      1109418|
|               Apple|       742633|
|              Huawei|       555636|
|             Generic|       176737|
|                  LG|       173070|
|     Generic_Android|       140957|
|                Sony|        78339|
|                 HTC|        74043|
|              Lenovo|        59090|
|              XiaoMi|        54380|
|Generic_Android_T...|        47866|
|               Nokia|        38581|
|                Asus|        29729|
|        SonyEricsson|        14810|
|            Micromax|         5022|
|            Motorola|         3172|
|                 ZTE|         2346|
|               Tecno|         2262|
+--------------------+-------------+
only showing top 20 rows



In [24]:
df_Devices_Actions = df_Devices_.withColumn('All_Actions', F.sum('Count_Actions').over(Window.partitionBy()))\
            .withColumn('Ratio_Actions', F.round(F.col('Count_Actions') / F.col('All_Actions'), 5))\
            .select('Device', 'Count_Actions', 'Ratio_Actions')
       
df_Devices_Actions.show()

23/01/23 21:33:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-------------+-------------+
|              Device|Count_Actions|Ratio_Actions|
+--------------------+-------------+-------------+
|             Samsung|      1845565|      0.17805|
|              Spider|      1109418|      0.10703|
|               Apple|       742633|      0.07165|
|              Huawei|       555636|      0.05361|
|             Generic|       176737|      0.01705|
|Generic_Android_T...|        47866|      0.00462|
|        SonyEricsson|        14810|      0.00143|
|            Motorola|         3172|       3.1E-4|
|               Tecno|         2262|       2.2E-4|
|           Microsoft|         1302|       1.3E-4|
|           Prestigio|          565|       5.0E-5|
|              Google|          408|       4.0E-5|
|                Acer|          260|       3.0E-5|
|               Umeox|          259|       2.0E-5|
|                  HP|          214|       2.0E-5|
|                vivo|          179|       2.0E-5|
|               Cubot|         

# Соединение таблиц 1 и 2

In [25]:
df_Devices_About = df_Devices_Users.join(df_Devices_Actions, ["Device"])
df_Devices_About.show()

23/01/23 21:34:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/23 21:34:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-----------+-----------+-------------+-------------+
|              Device|Count_Users|Ratio_Users|Count_Actions|Ratio_Actions|
+--------------------+-----------+-----------+-------------+-------------+
|                Acer|        260|     3.0E-5|          260|       3.0E-5|
|             Alcatel|        975|     9.0E-5|          975|       9.0E-5|
|              Amazon|          4|        0.0|            4|          0.0|
|               Aoson|         27|        0.0|           27|          0.0|
|               Apple|     742633|    0.07165|       742633|      0.07165|
|              Archos|         66|     1.0E-5|           66|       1.0E-5|
|                Asus|      29729|    0.00287|        29729|      0.00287|
|          BlackBerry|        545|     5.0E-5|          545|       5.0E-5|
|                 Blu|          1|        0.0|            1|          0.0|
|             Coolpad|         32|        0.0|           32|          0.0|
|               Cubot|   

# Создание суррогатного ключа

In [26]:
df_Devices_About = df_Devices_About.withColumn('original_order', F.monotonically_increasing_id())
df_Devices_About = df_Devices_About.withColumn('id_device', F.row_number().over(Window.orderBy('original_order')))
df_Devices_About = df_Devices_About.drop('original_order')
df_Devices_About.show()

23/01/23 21:35:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/23 21:35:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/23 21:35:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-----------+-----------+-------------+-------------+---------+
|              Device|Count_Users|Ratio_Users|Count_Actions|Ratio_Actions|id_device|
+--------------------+-----------+-----------+-------------+-------------+---------+
|                Acer|        260|     3.0E-5|          260|       3.0E-5|        1|
|             Alcatel|        975|     9.0E-5|          975|       9.0E-5|        2|
|              Amazon|          4|        0.0|            4|          0.0|        3|
|               Aoson|         27|        0.0|           27|          0.0|        4|
|               Apple|     742633|    0.07165|       742633|      0.07165|        5|
|              Archos|         66|     1.0E-5|           66|       1.0E-5|        6|
|                Asus|      29729|    0.00287|        29729|      0.00287|        7|
|          BlackBerry|        545|     5.0E-5|          545|       5.0E-5|        8|
|                 Blu|          1|        0.0|            1|     

# Сохранение результирующего дата фрейма "Характеристики устройств" в файл

In [27]:
df_Devices_About.toPandas().to_csv('Devices_About.txt',mode='w+',header=True,index=False)

23/01/23 21:36:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/23 21:36:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/23 21:36:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


# Формирование таблицы 3 "Браузеры"

In [28]:
df_Devices_Browsers = df_Base.groupby('Device','Browser')\
                             .agg(F.count('Browser').alias('Count_Browser'))
df_Devices_Browsers.show()

+---------------+--------------------+-------------+
|         Device|             Browser|Count_Browser|
+---------------+--------------------+-------------+
|          other|    Samsung Internet|         2703|
|         Spider|            heritrix|           20|
|          Apple|   Chrome Mobile iOS|        73520|
|          other|             Maxthon|         4444|
|          Apple|         Firefox iOS|         7127|
|   SonyEricsson|Chrome Mobile Web...|          315|
|          Nokia|              Google|           73|
|          Apple|            Applebot|        21783|
|          other|       GooglePlusBot|           77|
|            HTC|               Other|           33|
|         Haipad|             Android|            2|
|             LG|        Opera Mobile|          825|
|           Asus|        Opera Mobile|           37|
|         Huawei|Chrome Mobile Web...|        38301|
|         Spider|            Qwantify|          110|
|          ionik|              Google|        

In [30]:
window = Window.partitionBy(df_Devices_Browsers['Device'])\
               .orderBy(df_Devices_Browsers['Count_Browser'].desc())
df_Browsers = df_Devices_Browsers.select('*', F.row_number().over(window).alias('row_number')).where(F.col('row_number') <= 5)\
             .select('Device', 'Browser', 'Count_Browser', 'row_number')\
             .orderBy(df_Devices_Browsers['Device'],df_Devices_Browsers['Count_Browser'].desc())
df_Browsers.show()

+-------+--------------------+-------------+----------+
| Device|             Browser|Count_Browser|row_number|
+-------+--------------------+-------------+----------+
|   Acer|              Chrome|          223|         1|
|   Acer|             Android|           33|         2|
|   Acer|Chrome Mobile Web...|            3|         3|
|   Acer|              Google|            1|         4|
|Alcatel|             Android|          568|         1|
|Alcatel|       Chrome Mobile|          279|         2|
|Alcatel|              Chrome|          126|         3|
|Alcatel|              Google|            2|         4|
| Amazon|         Amazon Silk|            4|         1|
|  Aoson|Chrome Mobile Web...|           24|         1|
|  Aoson|             Android|            3|         2|
|  Apple|       Mobile Safari|       441774|         1|
|  Apple|   Chrome Mobile iOS|        73520|         2|
|  Apple|        MobileSafari|        69822|         3|
|  Apple|              Google|        43072|    

# Сохранение результирующего дата фрейма "Браузеры" в файл

In [31]:
df_Browsers.toPandas().to_csv('Browsers.txt',mode='w+',header=True,index=False)

# Формирование таблицы 4 "Ответы сервера"

In [32]:
df_Status_200 = df_Base.filter(df_Base.Status != "200")
df_Status_200.show()

+-------------+--------------------+--------------------+------+-----+--------------------+-------------+------+
|           IP|         Date_access|              Action|Status| Size|          User_agent|      Browser|Device|
+-------------+--------------------+--------------------+------+-----+--------------------+-------------+------+
|207.46.13.136|[22/Jan/2019:03:5...|GET /product/1492...|   404|33617|Mozilla/5.0 (comp...|      bingbot|Spider|
|66.249.66.194|[22/Jan/2019:03:5...|GET /product/8190...|   404|32278|Mozilla/5.0 (comp...|    Googlebot|Spider|
|  5.211.97.39|[22/Jan/2019:03:5...|HEAD /amp_preconn...|   404|    0|Mozilla/5.0 (iPho...|Mobile Safari| Apple|
| 54.36.148.87|[22/Jan/2019:03:5...|GET /filter/p65%2...|   302|    0|Mozilla/5.0 (comp...|    AhrefsBot|Spider|
|207.46.13.104|[22/Jan/2019:03:5...|GET /browse/flute...|   404|33605|Mozilla/5.0 (comp...|      bingbot|Spider|
| 5.160.157.20|[22/Jan/2019:03:5...|GET /browse/blu-r...|   301|  178|Mozilla/5.0 (Wind...|     

In [41]:
df_Status = df_Status_200.groupby('Device')\
             .agg(F.count('Status').alias('Count_Status_all'))\
             .withColumn('Count_Status_all',F.col('Count_Status_all'))        
df_Status.show()

+--------------------+----------------+
|              Device|Count_Status_all|
+--------------------+----------------+
|             Infinix|               7|
|               Nokia|            1262|
|              Spider|          393645|
|                 RCA|               1|
|                Sony|            3018|
|               Cubot|               3|
|             Alcatel|             114|
|            Motorola|              67|
|              XiaoMi|            1506|
|              Huawei|           18805|
|            Trekstor|               1|
|                  HP|              16|
|        SonyEricsson|             928|
|             Hyundai|               1|
|Generic_Android_T...|            1839|
|           Prestigio|              54|
|          BlackBerry|              68|
|               other|          179173|
|                Acer|               8|
|                Asus|            1039|
+--------------------+----------------+
only showing top 20 rows



In [36]:
df_Status_Answers = df_Status_200.groupby('Device','Status')\
             .agg(F.count('Status').alias('Count_Status'))\
             .withColumn('Count_Status',F.col('Count_Status'))\
             .orderBy('Device', 'Count_Status', ascending = False)\
             .select('Device', 'Status', 'Count_Status')
df_Status_Answers.show()

+------+------+------------+
|Device|Status|Count_Status|
+------+------+------------+
|  vivo|   302|           4|
| other|   302|       80452|
| other|   304|       32065|
| other|   499|       25256|
| other|   301|       20081|
| other|   500|       14090|
| other|   404|        5595|
| other|   400|         556|
| other|   502|         359|
| other|   401|         307|
| other|   403|         285|
| other|   408|          59|
| other|   504|          42|
| other|   414|          17|
| other|   405|           6|
| other|   206|           3|
|   ZTE|   302|          18|
|   ZTE|   499|           4|
|   ZTE|   301|           2|
|   ZTE|   404|           1|
+------+------+------------+
only showing top 20 rows



In [44]:
df_Server_Answers = df_Status.join(df_Status_Answers, ["Device"], "left")\
                    .orderBy('Device', 'Count_Status', ascending = True)
df_Server_Answers.show()

+-------+----------------+------+------------+
| Device|Count_Status_all|Status|Count_Status|
+-------+----------------+------+------------+
|   Acer|               8|   404|           1|
|   Acer|               8|   302|           7|
|Alcatel|             114|   500|           3|
|Alcatel|             114|   304|           3|
|Alcatel|             114|   403|           5|
|Alcatel|             114|   499|           8|
|Alcatel|             114|   302|          33|
|Alcatel|             114|   404|          62|
| Amazon|               1|   301|           1|
|  Apple|           99340|   400|           1|
|  Apple|           99340|   408|           8|
|  Apple|           99340|   401|          16|
|  Apple|           99340|   502|          34|
|  Apple|           99340|   500|          92|
|  Apple|           99340|   403|         355|
|  Apple|           99340|   304|        2366|
|  Apple|           99340|   301|        2893|
|  Apple|           99340|   499|        3850|
|  Apple|    

# Сохранение результирующего дата фрейма "Ответы сервера" в файл

In [45]:
df_Server_Answers.toPandas().to_csv('Server_Answers.txt',mode='w+',header=True,index=False)